In [41]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
import os
import json
import shapely
from shapely.geometry import box
import numpy as np

In [6]:
os.chdir('/anvil/projects/x-cis250634/team5')
print("Current Working Directory:", os.getcwd())

Current Working Directory: /anvil/projects/x-cis250634/team5


In [9]:
points = gpd.read_file('./shp/filtered_flooding_points_over_one_day_nopr.shp')

points_copy = points.copy()
if points_copy.crs != "EPSG:5070":
    points_copy = points_copy.to_crs("EPSG:5070")
    print(points_copy)
    print(points_copy.crs)

     latitude_d  longitude_   latitude   longitude               eventName  \
0      42.95753   -91.62403   42.95753   -91.62403    2016 August IA Flood   
1      42.99784   -91.65801   42.99784   -91.65801    2016 August IA Flood   
2      42.99799   -91.65846   42.99799   -91.65846    2016 August IA Flood   
3     42.957825  -91.623983  42.957825  -91.623983    2016 August IA Flood   
4     42.957825  -91.623983  42.957825  -91.623983    2016 August IA Flood   
...         ...         ...        ...         ...                     ...   
1142  36.322586  -95.990324  36.322586  -95.990324  Central US Spring 2019   
1143  36.304303    -95.9913  36.304303    -95.9913  Central US Spring 2019   
1144  36.371158  -95.996168  36.371158  -95.996168  Central US Spring 2019   
1145   36.30578   -95.99089   36.30578   -95.99089  Central US Spring 2019   
1146   36.30207    -95.9921   36.30207    -95.9921  Central US Spring 2019   

      hwmTypeNam              hwmQuality verticalDa  \
0      S

In [48]:
# Find the temporally aligned images with the point
with open('./data_0718/json/flooding_dataset_with_precipitation_0801_with_height_above.json') as f:
    aligned_data = json.load(f)
    
    for data in aligned_data:
        # print(f"data id: {data['id']}")
        point = points_copy[points_copy['ID'] == str(data['id'])]
        # # small buffer, e.g., 3000 meters
        # if not point.empty:
        #     geom = point.geometry.buffer(3000).values[0]
        #     geojson_geom = [geom.__geo_interface__]
        # point_buffer = point.geometry.buffer(500)
        prec = data['CHIRPS_1day_ahead']
        prec_date = os.path.splitext(data['CHIRPS_1day_ahead'])[0]
        dst_crs = "EPSG:5070"
        
        with rasterio.open(f'./data_0718/CHIRPS/CHIRPS_1day_ahead/{prec}') as src:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })
        
            # with rasterio.open("/xchen/reprojected_raster.tif", 'w', **kwargs) as dst:
            #     for i in range(1, src.count + 1):  # loop through all bands
            #         reproject(
            #             source=rasterio.band(src, i),
            #             destination=rasterio.band(dst, i),
            #             src_transform=src.transform,
            #             src_crs=src.crs,
            #             dst_transform=transform,
            #             dst_crs=dst_crs,
            #             resampling=Resampling.nearest  # or bilinear, cubic, etc.
            #         )
                
            # with rasterio.open("reprojected_raster.tif", 'r') as dst:
            #     # if geom.intersects(box(*dst.bounds)):
            #     #     out_image, out_transform = mask(dst, geojson_geom, crop=True)
            #     #     precipitation = out_image[out_image != dst.nodata].mean()
            #     # else:
            #     #     coords = [(x,y) for x, y in zip(point.geometry.x, point.geometry.y)]
            #     #     values = list(dst.sample(coords))
   
            #     #     precipitation = [v[0] for v in values]
            #     coords = [(x,y) for x, y in zip(point.geometry.x, point.geometry.y)]
            #     values = list(dst.sample(coords))
            #     precipitation = [v[0] for v in values]
            #     if precipitation is not None:
            #         # print(precipitation)
            #         points_copy.loc[points_copy['ID'] == str(data['id']), 'precipitation'] = precipitation
            dst_array = np.empty((src.count, height, width), dtype=src.dtypes[0])
            with MemoryFile() as memfile:
                with memfile.open(driver='GTiff',
                                  height=height,
                                  width=width,
                                  count=src.count,
                                  dtype=src.dtypes[0],
                                  crs=dst_crs,
                                  transform=transform) as dst:
                    for i in range(src.count):
                        dst.write(dst_array[i], i + 1)
            
                    coords = [(x,y) for x, y in zip(point.geometry.x, point.geometry.y)]
                    values = list(dst.sample(coords))
                    precipitation = [v[0] for v in values]
                    if precipitation is not None:
                        # print(precipitation)
                        points_copy.loc[points_copy['ID'] == str(data['id']), 'precipitation'] = precipitation
                        points_copy.loc[points_copy['ID'] == str(data['id']), 'prec_date'] = prec_date
print("Precipitation added!")

Precipitation added!


In [44]:
print("Precipitation:\n", points_copy['precipitation'])
points_copy[['ID', 'S1_Date', 'peak_date', 'precipitation']].to_csv('./xchen/precipitation.csv', index=False)

Precipitation:
 0       0.000000e+00
1       0.000000e+00
2       0.000000e+00
3       0.000000e+00
4       0.000000e+00
            ...     
1142    5.680864e-41
1143    1.238019e-40
1144    1.796212e-40
1145   -4.220406e-29
1146    2.939083e-40
Name: precipitation, Length: 1147, dtype: float32


In [50]:
soil_moisture_tifs = ['Flooding_SMAP9KM_3hr_before', 'Flooding_SMAP9KM_6hr_before', 'Flooding_SMAP9KM_9hr_before', 'Flooding_SMAP9KM_12hr_before',]

In [59]:
with open('./data_0718/json/flooding_dataset_with_precipitation_0801_with_height_above.json') as f:
    aligned_data = json.load(f)
    
    for data in aligned_data:
        # print(f"data id: {data['id']}")
        point = points_copy[points_copy['ID'] == str(data['id'])]
        # point_buffer = point.geometry.buffer(500)
        soil = data['SMAP9KM_3hr_before']
        dst_crs = "EPSG:5070"
        
        for soil_moisture_tif in soil_moisture_tifs:
            soil_moi_max = []
            if os.path.exists(f'./data_0718/Flooding_SMAP9KM/{soil_moisture_tif}/{soil}'):
                print(f'./data_0718/Flooding_SMAP9KM/{soil_moisture_tif}/{soil}')
                with rasterio.open(f'./data_0718/Flooding_SMAP9KM/{soil_moisture_tif}/{soil}') as src:
                    transform, width, height = calculate_default_transform(
                        src.crs, dst_crs, src.width, src.height, *src.bounds)
                    
                    kwargs = src.meta.copy()
                    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })
                
                    with rasterio.open("reprojected_raster.tif", 'w', **kwargs) as dst:
                        for i in range(1, src.count + 1):  # loop through all bands
                            reproject(
                                source=rasterio.band(src, i),
                                destination=rasterio.band(dst, i),
                                src_transform=src.transform,
                                src_crs=src.crs,
                                dst_transform=transform,
                                dst_crs=dst_crs
                                # resampling=Resampling.nearest
                            )
                        
                    with rasterio.open("reprojected_raster.tif", 'r') as dst:
                        coords = [(x,y) for x, y in zip(point.geometry.x, point.geometry.y)]
                        values = list(dst.sample(coords))
                        soil_moisture = [v[0] for v in values]
                    
                        # Assign directly to `points` using .loc
                        if soil_moisture is not None:
                            soil_moi_max.append(soil_moisture)

        points_copy.loc[points_copy['ID'] == data['id'], 'soil_moisture'] = max(soil_moi_max)
    # print("Precipitation:\n", points_copy['precipitation'])

./data_0718/Flooding_SMAP9KM/Flooding_SMAP9KM_3hr_before/770_2016-08-25T09:00:00_SMAP.tif


ValueError: max() iterable argument is empty

In [ ]:
print("soil_moisture:\n", points_copy['soil_moisture'])
points_copy[['ID', 'S1_Date', 'soil_moisture']].to_csv('./xchen/soil_moisture.csv', index=False)